In [6]:
import librosa
import sounddevice as sd
import numpy as np
import tensorflow as tf

In [15]:
model_pth = r"D:\model_code\server_models\mobile_net\trail_1\_1\old_samples\16k_melspec-nfft-1024_a_h_cnn_dense_model.keras"

In [16]:
model = tf.keras.models.load_model(model_pth)

In [5]:
fs = 44100
sr = 16000
buffer_duration = 2 # seconds
buffer_size = buffer_duration * fs

In [10]:
label = {
    0:'unknown',1:'adele',2:'hilfe'
}
pad_or_trunc = lambda a,i : a[0:i] if len(a) > i else a if len(a) == i else np.pad(a,(0, (i-len(a))))
max_seconds = 1
target_sr = 16000

def process_data(y,sr,max_seconds):
    y = pad_or_trunc(y,sr*max_seconds)
    features = librosa.feature.melspectrogram(y=y,sr=sr,n_fft=1024)
    return features

def get_features(waveform, sr):
    features = process_data(waveform,sr,max_seconds)
    shape = features.shape
    features = features.reshape([1,*shape,1])
    return features

def predict(data,sr):
    # data,_ = librosa.load(data,sr=sr)
    y = librosa.resample(data,orig_sr=sr,target_sr=target_sr)
    features = get_features(y,sr=target_sr)
    pred = model.predict(features)
    return label.get(np.argmax(pred)),np.max(pred)
    return y,(label.get(np.argmax(pred) if np.max(pred) >= 0.97 else 0,'unknown'),round(np.max(pred)*100,2))

In [19]:
while True:
    try:
        # print("Recording...")
        # Record audio for 2 seconds
        buffer = sd.rec(buffer_size, channels=1, dtype='float32')
        sd.wait()  # Wait until recording is finished
        sd.play(buffer,fs)
        sd.wait()  # Wait until recording is finished
        # Process the recorded audio
        y = buffer.squeeze()[fs:]
        # data = librosa.resample(y=y,orig_sr=fs,target_sr=sr)
        l,p = predict(data,fs)
        print(l,p or '',end=' ')
        # break
        # clear_output(wait=True)
    except KeyboardInterrupt:
        print("Stream stopped by user")
        break

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
unknown 0.6881904 Stream stopped by user
